In [526]:
# import autograd functionally
import autograd.numpy as np
from autograd.misc.flatten import flatten_func
from autograd import grad as compute_grad

# import various other libraries
import copy
import matplotlib.pyplot as plt

# this is needed to compensate for %matplotl+ib notebook's tendancy lotted inline
from matplotlib import rcParams
rcParams["figure.autolayout"] = True

In [527]:
# gradient descent function
def gradient_descent(g, w ,alpha, max_its, beta):
    # flatten the input function, create gradient based on flat function
    g_flat, unflatten, w = flatten_func(g, w)
    grad = compute_grad(g_flat)

    # record history
    w_hist = []
    # push the first w
    w_hist.append(unflatten(w))

    # start gradient descent loop
    z = np.zeros(np.shape(w))  # momentum term

    # over the line
    for k in range(max_its):
        # plug in value into func and derivative
        grad_eval = grad(w)
        grad_eval.shape = np.shape(w)

        # take descent step with momentum
        z = beta * z + grad_eval
        w = w -alpha * z

        # record weight update
        w_hist.append(unflatten(w))

    return w_hist

In [528]:
def plotting(cost_hist, ylabelName, label):
    figure, axes = plt.subplots(1,1, figsize = (6,5))
    axes.plot(range(len(cost_hist)),cost_hist, label = label, linestyle = "solid")
    axes.set_xlabel("iterations")
    axes.set_ylabel(ylabelName)
    axes.legend()
    plt.show()

def model(x ,w):
    # feature transformations
    f = w[0] + np.dot(x, w[1:])
    return f

def statistics_number(y, alpha = 1):
    negative = np.array(y == -1, dtype = np.int)
    positive = np.array(y == 1,dtype = np.int)
    nega_number = np.sum(negative)
    posi_number = np.sum(positive)
    if nega_number < posi_number:
        nega_coefficient = negative * alpha
        posi_coefficient = positive * 1.0
    else:
        nega_coefficient = negative * 1.0
        posi_coefficient = positive * alpha      
    coefficient = nega_coefficient + posi_coefficient 
    return coefficient

In [529]:
csvname = 'imbalanced_2class.csv'
data = np.loadtxt(csvname, delimiter = ',')
x = data[:,:-1]
y = data[:,-1:]

In [530]:
beta = statistics_number(y,alpha = 1)

In [531]:
def sign(x):
    a_1 = np.array(x>=0, dtype = np.int)
    a_2 = np.array(x<0, dtype = np.int)
    return a_1*1 + a_2*(-1)

def softmax_cost(w):
    cost = np.sum(beta*np.log(1+np.exp(-y*model(x,w))))
    return cost/float(np.size(y))

In [532]:
# initialize parameters
alpha = 0.2
max_its = 500
gamma = 0
w_init = np.random.randn(x.shape[1]+1, 1)

In [533]:
beta = statistics_number(y,alpha = 1)
# run gradient descent, create cost function history
weight_history = gradient_descent(softmax_cost, w_init, alpha, max_its,beta = gamma)
# use MSE to validate the regression quality
cost_history = [softmax_cost(v) for v in weight_history]
best_w = weight_history[-1]

In [534]:
from sklearn.metrics import confusion_matrix
y_true = data[:,-1]

In [535]:
def accuracy(w):
    y_pred = sign(model(x,w))
    y_pred = [y_pred[i][0] for i in range(np.size(y_pred))]
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()
    A_positive = tp/(tp+fn)
    A_negative = tn/(tn+fp)
    balanced_A = 0.5*(A_positive+A_negative)
    accuracy = (tn+tp)/(tn + fp + fn + tp)
    return accuracy, balanced_A
    

In [536]:
accuracy_1, balanced_accuracy = accuracy(best_w)
print("accuracy: ",accuracy_1, "balanced_accuracy: ",balanced_accuracy)

accuracy:  0.9 balanced_accuracy:  0.5


In [537]:
beta = statistics_number(y,alpha = 5)
# run gradient descent, create cost function history
weight_history = gradient_descent(softmax_cost, w_init, alpha, max_its,beta = gamma)
# use MSE to validate the regression quality
cost_history = [softmax_cost(v) for v in weight_history]
best_w = weight_history[-1]

In [538]:
accuracy_2, balanced_accuracy = accuracy(best_w)
print("accuracy: ",accuracy_2, "balanced_accuracy: ",balanced_accuracy)

accuracy:  1.0 balanced_accuracy:  1.0
